In [1]:
#!fsharp
#!value --from-file input.txt --name input

In [1]:
#!fsharp
#!share input --from value
input

let parse (str:string) =
  str.Trim().Split() |> array2D

let seatMap = parse input

let neighbors array x y =
  seq {
    for i in (max 0 (x-1))..(min (Array2D.length1 array - 1) (x+1)) do
      for j in (max 0 (y-1))..(min (Array2D.length2 array - 1) (y+1)) do
        if i <> x || j <> y then 
          yield array.[i,j]
  }

let nextGen array x y =
  match Array2D.get array x y with
  | 'L' when neighbors array x y |> Seq.forall ((<>) '#') -> '#'
  | '#' when (neighbors array x y |> Seq.filter ((=) '#') |> Seq.length) >= 4 -> 'L'
  | x -> x

let evolve f array =
  array |> Array2D.mapi (fun x y _ -> f array x y)

let evolveFully f = 
  Seq.unfold (fun s -> 
    let next = evolve f s
    if s = next then None else Some (next, next)
  ) >> Seq.last

let countAll c =
  Seq.cast<char> 
  >> Seq.filter ((=) c)
  >> Seq.length

let example = """L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL"""

let exampleMap = example.Trim().Split() |> array2D

seatMap
|> evolveFully nextGen 
|> countAll '#'

2412

In [1]:
#!fsharp
let visibleNeighbors array x y =
  let rec firstSeat (i,j) a b = 
    let p, q = a + i, b + j
    if p < 0 || q < 0 || p >= Array2D.length1 array || q >= Array2D.length2 array then '.' else
    match array.[p, q] with
    | '.' -> firstSeat (i,j) p q
    | x -> x
  seq {
    for i in -1..1 do
      for j in -1..1 do
        if i <> 0 || j <> 0 then 
          yield firstSeat (i,j) x y
  }

let nextGen2 array x y =
  match Array2D.get array x y with
  | 'L' when visibleNeighbors array x y |> Seq.forall ((<>) '#') -> '#'
  | '#' when (visibleNeighbors array x y |> Seq.filter ((=) '#') |> Seq.length) >= 5 -> 'L'
  | x -> x

seatMap
|> evolveFully nextGen2
|> countAll '#'


2176